In [18]:
import os
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
import pandas as pd

### SCOPES: 

Это переменная, которая определяет разрешения (scopes) для доступа к Google Sheets. 

### SPREADSHEED_ID: 

Это переменная, которая хранит уникальный идентификатор (ID) для конкретной таблицы Google Sheets. 

In [2]:
SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
SPREADSHEED_ID = '1qu1g_wCtd09UIudJ01dpMnrFFVVJQsoQFsjipSRlPso'

 ### Получению учетных данных (credentials) для взаимодействия с Google Sheets через Google API

In [25]:
credentials = None

if os.path.exists('token.json'):
    credentials = Credentials.from_authorized_user_file('token.json', SCOPES)

if not credentials or not credentials.valid:
    if credentials and credentials.expired and credentials.refresh_token:
        credentials.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file('creds.json', SCOPES)
        credentials = flow.run_local_server(port=0)
    with open('token.json', 'w') as token:
        token.write(credentials.to_json())

### Контент Google Sheets до тестовых операций

In [27]:
try: 
    service = build('sheets', 'v4', credentials=credentials)
    sheets = service.spreadsheets()
    
    # initial values
    result = sheets.values().get(spreadsheetId = SPREADSHEED_ID, range='Sheet1').execute()
    values = result.get('values', [])
    print(pd.DataFrame(values))


except HttpError as er:
    print(er)

      0     1    2         3
0  num1  num2  sum      need
1     1     1       Not Done
2     2     2       Not Done
3     3     3       Not Done
4     4     4       Not Done
5     5     5       Not Done


## Тестовые операции:

**num1** = получает значение из столбца A для текущей строки.

**num2** = получает значение из столбца B для текущей строки.

**sum_res** = сумма значений из столбца A и B для текущей строки, который отправляется как новое значение суммы в ячейку C текущей строки.

In [35]:
try: 
    service = build('sheets', 'v4', credentials=credentials)
    sheets = service.spreadsheets()
    
    for row in range(2,7):
        num1 = int(sheets.values().get(spreadsheetId = SPREADSHEED_ID, range=f'Sheet1!A{row}').execute().get('values')[0][0])
        num2 = int(sheets.values().get(spreadsheetId = SPREADSHEED_ID, range=f'Sheet1!B{row}').execute().get('values')[0][0])
        print(num1, num2, sum_res := num1+num2)

        sheets.values().update(spreadsheetId = SPREADSHEED_ID, range=f'Sheet1!C{row}', valueInputOption="USER_ENTERED", body={'values': [[f'{sum_res}']]}).execute()
        
        sheets.values().update(spreadsheetId = SPREADSHEED_ID, range=f'Sheet1!D{row}', valueInputOption="USER_ENTERED", body={'values': [['Done']]}).execute()

except HttpError as er:
    print(er)

1 1 2
2 2 4
3 3 6
4 4 8
5 5 10


### Контент Google Sheets после тестовых операций

In [36]:
try: 
    service = build('sheets', 'v4', credentials=credentials)
    sheets = service.spreadsheets()
    
    # initial values
    result = sheets.values().get(spreadsheetId = SPREADSHEED_ID, range='Sheet1').execute()
    values = result.get('values', [])
    print(pd.DataFrame(values))


except HttpError as er:
    print(er)

      0     1    2     3
0  num1  num2  sum  need
1     1     1    2  Done
2     2     2    4  Done
3     3     3    6  Done
4     4     4    8  Done
5     5     5   10  Done
